# Direct Openings

In [75]:
import json
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import load_model
import numpy as np

model = load_model('../../../models/openings/recommendations.h5')

df = pd.read_csv('../../../data/opening_scores.csv')

with open('../../../models/openings/user_embeddings.json') as f:
    user_embeddings = json.load(f)
with open('../../../models/openings/opening_embeddings.json') as f:
    opening_embeddings = json.load(f)
with open('../../../models/openings/user_bias_embeddings.json') as f:
    user_bias_embeddings = json.load(f)
with open('../../../models/openings/opening_bias_embeddings.json') as f:
    opening_bias_embeddings = json.load(f)

In [76]:
def predict(user_id, opening_id):
    user_embedding = np.array(user_embeddings[user_id])
    opening_embedding = np.array(opening_embeddings[opening_id])

    # Passing user embedding and movie embedding through the concat layer
    concatenated_embeddings = model.get_layer('concat')([user_embedding, opening_embedding])

    # Passing the concatenated embeddings through the dense layers
    x = model.get_layer('dense1')(concatenated_embeddings)
    x = model.get_layer('dense2')(x)

    user_bias_embedding = np.array(user_bias_embeddings[user_id])
    opening_bias_embedding = np.array(opening_bias_embeddings[opening_id])

    user_bias_embedding = user_bias_embedding[:, tf.newaxis]
    opening_bias_embedding = opening_bias_embedding[:, tf.newaxis]

    # Combine embeddings, biases, and pass through the output layer
    input_tensors = [x, user_bias_embedding, opening_bias_embedding]
    concatenated_features = model.get_layer('combined_features')(input_tensors)
    
    x = model.get_layer('combined_dense1')(concatenated_features)
    x = model.get_layer('combined_dense2')(x)
    
    x = model.get_layer('output')(x)

    predicted_rating = x[0][0]

    return predicted_rating.numpy()

In [77]:
def direct_user_recommendations(user_id):
    user_ratings = df[df['user_id'] == user_id]
    user_ratings = user_ratings[user_ratings['score'] != 1]
    
    recommendation = df[~df['opening_id'].isin(user_ratings['opening_id'])][['opening_id']].drop_duplicates()
    recommendation['score_predict'] = recommendation.apply(lambda x: predict(user_id, x['opening_id']), axis=1)
    
    recomendations = {}

    def df2dict(row):
        recomendations[row['opening_id']]=row['score_predict']
        
    recommendation.apply(df2dict, axis=1)
    return recomendations

In [80]:
direct_user_recommendations('5f5a90e2-6601-480c-bc08-7592e06beacd')

{'d569ec70-f3fa-421c-88db-0773b26a2475': 3.5413594245910645,
 '1067c159-be01-4f89-a1cf-a32cdd67bb72': 3.5784387588500977}

# Through Projects

In [82]:
df = pd.read_csv('../../../data/openings.csv')

model = load_model('../../../models/projects/recommendations.h5')

with open('../../../models/projects/user_embeddings.json') as f:
    user_embeddings = json.load(f)
with open('../../../models/projects/project_embeddings.json') as f:
    project_embeddings = json.load(f)
with open('../../../models/projects/user_bias_embeddings.json') as f:
    user_bias_embeddings = json.load(f)
with open('../../../models/projects/project_bias_embeddings.json') as f:
    project_bias_embeddings = json.load(f)

def predict_score(user_id, project_id):
    user_embedding = np.array(user_embeddings[user_id])
    project_embedding = np.array(project_embeddings[project_id])

    # Passing user embedding and movie embedding through the concat layer
    concatenated_embeddings = model.get_layer('concat')([user_embedding, project_embedding])

    # Passing the concatenated embeddings through the dense layers
    x = model.get_layer('dense1')(concatenated_embeddings)
    x = model.get_layer('dense2')(x)

    user_bias_embedding = np.array(user_bias_embeddings[user_id])
    project_bias_embedding = np.array(project_bias_embeddings[project_id])

    user_bias_embedding = user_bias_embedding[:, tf.newaxis]
    project_bias_embedding = project_bias_embedding[:, tf.newaxis]

    # Combine embeddings, biases, and pass through the output layer
    input_tensors = [x, user_bias_embedding, project_bias_embedding]
    concatenated_features = model.get_layer('combined_features')(input_tensors)
    
    x = model.get_layer('combined_dense1')(concatenated_features)
    x = model.get_layer('combined_dense2')(x)
    
    x = model.get_layer('output')(x)

    predicted_rating = x[0][0]
    return predicted_rating.numpy()

def recommend_projects(userID):
    df = pd.read_csv('../../../data/project_scores.csv')
    user_ratings = df[df['user_id'] == userID]
    user_ratings = user_ratings[user_ratings['score'] != 1]
    recommendation = df[~df['project_id'].isin(user_ratings['project_id'])][['project_id']].drop_duplicates()
    recommendation['score_predict'] = recommendation.apply(lambda x: predict_score(userID, x['project_id']), axis=1)
    
    final_rec = recommendation.sort_values(by='score_predict', ascending=False)
    return final_rec['project_id'].values.tolist()

In [83]:
import tensorflow as tf

with open('../../../models/openings/user_embeddings.json') as f:
    user_embeddings = json.load(f)
with open('../../../models/openings/opening_embeddings.json') as f:
    opening_embeddings = json.load(f)
with open('../../../models/openings/user_bias_embeddings.json') as f:
    user_bias_embeddings = json.load(f)
with open('../../../models/openings/opening_bias_embeddings.json') as f:
    opening_bias_embeddings = json.load(f)

model = load_model('../../../models/openings/recommendations.h5')

def predict(user_id, opening_id):
    user_embedding = np.array(user_embeddings[user_id])
    opening_embedding = np.array(opening_embeddings[opening_id])

    # Passing user embedding and movie embedding through the concat layer
    concatenated_embeddings = model.get_layer('concat')([user_embedding, opening_embedding])

    # Passing the concatenated embeddings through the dense layers
    x = model.get_layer('dense1')(concatenated_embeddings)
    x = model.get_layer('dense2')(x)

    user_bias_embedding = np.array(user_bias_embeddings[user_id])
    opening_bias_embedding = np.array(opening_bias_embeddings[opening_id])

    user_bias_embedding = user_bias_embedding[:, tf.newaxis]
    opening_bias_embedding = opening_bias_embedding[:, tf.newaxis]

    # Combine embeddings, biases, and pass through the output layer
    input_tensors = [x, user_bias_embedding, opening_bias_embedding]
    concatenated_features = model.get_layer('combined_features')(input_tensors)
    
    x = model.get_layer('combined_dense1')(concatenated_features)
    x = model.get_layer('combined_dense2')(x)
    
    x = model.get_layer('output')(x)

    predicted_rating = x[0][0]

    return predicted_rating.numpy()

In [84]:
scores = pd.read_csv('../../../data/opening_scores.csv')
scores = scores.merge(df, left_on='opening_id', right_on='id',)

def through_user_recommendations(user_id):
    user_ratings = scores[scores['user_id'] == user_id]
    user_ratings = user_ratings[user_ratings['score'] != 1]
    
    recommended_projects=recommend_projects(user_id)
    
    recommendation = scores[~scores['opening_id'].isin(user_ratings['opening_id'])].drop_duplicates()
    recommendation = recommendation[recommendation['project_id'].isin(recommended_projects)][['opening_id']].drop_duplicates()
    recommendation['score_predict'] = recommendation.apply(lambda x: predict(user_id, x['opening_id']), axis=1)
    
    recomendations = {}

    def df2dict(row):
        recomendations[row['opening_id']]=row['score_predict']
        
    recommendation.apply(df2dict, axis=1)
    return recomendations

In [85]:
through_user_recommendations('5f5a90e2-6601-480c-bc08-7592e06beacd')

{'d569ec70-f3fa-421c-88db-0773b26a2475': 3.5413594245910645}

# Combined

In [81]:
userID = '5f5a90e2-6601-480c-bc08-7592e06beacd'

direct=direct_user_recommendations(userID)

In [86]:
through=through_user_recommendations(userID)

In [87]:
alpha = 0.7
beta = 0.3 

In [97]:
combined_recommendations = {}

for id in direct:
    if id not in combined_recommendations:
        combined_recommendations[id] = alpha * direct[id]

for id in through:
    if id not in combined_recommendations:
        combined_recommendations[id] = beta * through[id]
    else:
        combined_recommendations[id] += beta * through[id]

In [98]:
combined_recommendations=dict(sorted(combined_recommendations.items(), key=lambda item: item[1], reverse=True))

In [99]:
combined_recommendations

{'d569ec70-f3fa-421c-88db-0773b26a2475': 3.5413594245910645,
 '1067c159-be01-4f89-a1cf-a32cdd67bb72': 2.5049071311950684}

In [100]:
list(combined_recommendations.keys())

['d569ec70-f3fa-421c-88db-0773b26a2475',
 '1067c159-be01-4f89-a1cf-a32cdd67bb72']